[Cython]: http://cython.org/
[docs]: http://docs.cython.org/en/latest/src/quickstart/index.html
[wiki]: https://github.com/cython/cython/wiki

[tutorial]: http://conference.scipy.org/proceedings/SciPy2009/paper_1/full_text.pdf
[numerical calculations]: http://conference.scipy.org/proceedings/SciPy2009/paper_2/full_text.pdf
[tinyr]: https://code.google.com/archive/p/tinyr/


# Cython

We now will try [Cython] as a solution to speedup our RBush ([docs], [wiki]).
Cython is a superset of Python, where special syntax can be used to call C-level functions and to compile the \*ython code as C.
Cython claims that by using the cython compiler engine even pure-python code can benefit (even if slightly) from (use of C) optimizations.

Two documents should help us start this journey, the Cython [tutorial] by *S. Behnel, R.W. Bradshaw & D.S. Seljebotn* and a benchmark on [numerical calculations] done by *Dag Sverre Seljebotn*.
We also take [tinyr], an implementation of a r-tree, as our starting point.

If you're using Anaconda, Cython is available through the default channel for install.

[Sage]: http://www.sagemath.org/

## Cython tutorial

From the tutorial we learn the following.

Cython code can be compiled using:
* `setup.py` setup (what we will eventually use);
* `pyximport` to call cython' `pyx` files as if they were `py` modules and have compilation done in the background;
* pre-compile the code with `cython` command-line utility (most for debugging/tests);
* by using [Sage] notebooks, which allows Cython code inline (most for experimentation).



# Profiling

We're now at py-rbush commit #`6b48a144f0d602c90cc32d9c82771a131242fedc`, cython functions to `insert` and `search` have been adapted using pieces of `tinyr` and is now time to profile those functions.

The new version using `cython` is doing much better then our previous attempt with `numba` (whenever possible\*), but it still doing worse (~10x) than the brute-force\*\* method -- this one using all numba. Since it is a brute-force, and the purpose of using a tree structure is to reduced (logarithmically) the time consumed, we have to figure out where/what is going on and fix it until we get a substantial improvement (~100x?) to a typical use case (1M items).

To figure that out, we have to profile the functions in use and, probably, eventually we'll go down to line profiling.

\*: we couldn't use numba (version 0.36) everywhere in `rbush` because the library still has some limitations: it does not accept lists with non-homogeneous (*i.e*, different sizes) items, and when items were abstracted in (numba) classes the performance gain was quite low.

\*\*: the brute-force method is to search the items (boxes) linearly, one-by-one in our entire dataset.

## python --> cython

Python code is typically profiled using the stdlib `profile` or `cProfile` [profilers]. We can also easily inspect the performance of our code line-by-line using the [line_profiler] package.

Cython profiling is pretty close to a pure python code, we just have to add some pragmas to it according to how/what we want to inspect; their [profiling tutorial] gives us a nice first-steps guide.
Some googling on the subject inform us that the [line_profiler may be used with cython][so_lp].

So, here is the summary (so far, from what I got):
* we can use `cprofile` to profile an entire (`pyx`) module;
* we exclude some functions out of interest by decorating it accordingly;
* we can use line-tracing (with cython/cprofile);
* we can annotate the line-tracing with the `coverage` package;
* we can use Kern's `line_profiler` to have the timings per-line.

[profilers]: https://docs.python.org/3/library/profile.html
[line_profiler]: https://github.com/rkern/line_profiler
[profiling tutorial]: http://cython.readthedocs.io/en/latest/src/tutorial/profiling_tutorial.html
[so_lp]: https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line

## The simplest first

First step I will take is to simply enable the profiling using nothing more then the minimal settings.
This means we'll do as with a pure-python code: we use cProfile while running our test case.

The only addition is to include the pragma
```python
# cython: profile=True
```
to the top of the (`pyx`) module being profiled (file `core_funcs.pyx`).

In [1]:
import rbush
from rbush.data import generate_data_array

data = generate_data_array(10000)

r = rbush.RBush()
r.load(data)
dt = generate_data_array(1000)

In [2]:
def searchme(dt):
    for i in range(len(dt)):
        _ = r.search(*tuple(dt[i]))

In [3]:
import cProfile
import pstats

# import pyximport
# pyximport.install()

cProfile.runctx('searchme(dt)', globals(), locals(), 'profile.prof')

s = pstats.Stats('profile.prof')
s.strip_dirs().sort_stats('time').print_stats()

Thu Feb  8 00:10:11 2018    profile.prof

         2005 function calls in 2.725 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    2.705    0.003    2.705    0.003 {rbush.core_search.search}
        1    0.017    0.017    2.725    2.725 <ipython-input-2-06d109dea391>:1(searchme)
     1000    0.002    0.000    2.707    0.003 core.py:142(search)
        1    0.000    0.000    2.725    2.725 {built-in method builtins.exec}
        1    0.000    0.000    2.725    2.725 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Notice that we just profiled everything in our code.

If we want we can disable the functions out of our interest by decorating them with:
```python
cimport cython
@cython.profile(False)
def non_interesting_function():
    # ...
```

# Let's see what coverage is about

I understand that the line-tracing with the annotation done by `coverage` is helpful to see some sharp edges, let's check.

Now, instead of using `# cython: profile=True` we'll use `# cython: linetrace=True` at the top of our cython module.
Also, we have to add a `.coveragerc` file in our current directory with the content:
```
[run]
plugins = Cython.Coverage
```

Run the profiling script (above) and format that to a HTML for better visualization:
```bash
# cython  --annotate-coverage coverage.xml  rbush/core_search.pyx
```

In [4]:
from IPython.display import HTML
from IPython.display import display

#$ cython  --annotate-coverage coverage.xml  rbush/core_search.pyx
#display(HTML('core_search.html'))

# Line profiling

Line profiling (using `line_profile`) is a bit more complicated...not clear at all to me. But here goes a tentative after the docs:
* https://stackoverflow.com/questions/28301931/how-to-profile-cython-functions-line-by-line
* http://nbviewer.jupyter.org/gist/tillahoffmann/296501acea231cbdf5e7
* http://docs.cython.org/en/latest/src/reference/compilation.html
* http://blog.yclin.me/gsoc/2016/07/23/Cython-IPython/

In [5]:
#Load Robert Kern's line profiler
%load_ext line_profiler
import line_profiler

# Set compiler directives (cf. http://docs.cython.org/src/reference/compilation.html)
#
import Cython
directive_defaults = Cython.Compiler.Options.get_directive_defaults()
directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [6]:
%load_ext Cython

In [7]:
directive_defaults['wraparound'] = False
directive_defaults['boundscheck'] = False
directive_defaults['initializedcheck'] = False

In [23]:
%%cython -a -f --compile-args=-DCYTHON_TRACE=1

from cpython cimport array
import array

cimport numpy as np
np.import_array()

ctypedef np.int_t DTINT_t
ctypedef np.float_t DTFLT_t

import numpy as np

# @profile
def search_node(tuple node, np.ndarray[DTFLT_t] bbox, list items):
    cdef:
#         np.ndarray[DTFLT_t] ibox
        object ibox
        double xmin, ymin, xmax, ymax
        bint node_lower_bbox
        bint node_upper_bbox
        bint intersects
        tuple child

    xmin = bbox[0]
    ymin = bbox[1]
    xmax = bbox[2]
    ymax = bbox[3]
    
    ibox = node[0]
    # Considering 'x' the horizontal axes and 'y' the vertical..
    # * if bbox right border is at the right of node left border
    #      and bbox upper border is above node lower border
    # * if bbox left border is at the left of node right border
    #      and bbox lower border is below node upper border
    # * they intersect:
    node_lower_bbox = ibox[0] <= xmax and ibox[1] <= ymax
    node_upper_bbox = ibox[2] >= xmin and ibox[3] >= ymin
    intersects = node_lower_bbox and node_upper_bbox
    if not intersects:
        return

    if node[2]:
        items.extend(node[1])
        return

    for child in node[1]:
        search_node(child, bbox, items)


cdef list search_nodes(np.ndarray[DTFLT_t] bbox, np.ndarray[DTFLT_t, ndim=2] iboxes, np.ndarray[DTINT_t] data):
    cdef:
        np.ndarray[DTFLT_t] ibox
        unsigned int leng, i=0
        double xmin, ymin, xmax, ymax
        bint node_lower_bbox
        bint node_upper_bbox
        bint intersects
        array.array a = array.array('i')
        list out = []

    xmin = bbox[0]
    ymin = bbox[1]
    xmax = bbox[2]
    ymax = bbox[3]
    
    leng = len(iboxes)
    for i in range(leng):
        ibox = iboxes[i]
        # Considering 'x' the horizontal axes and 'y' the vertical..
        # * if bbox right border is at the right of node left border
        #      and bbox upper border is above node lower border
        # * if bbox left border is at the left of node right border
        #      and bbox lower border is below node upper border
        # * they intersect:
        node_lower_bbox = ibox[0] <= xmax and ibox[1] <= ymax
        node_upper_bbox = ibox[2] >= xmin and ibox[3] >= ymin
        intersects = node_lower_bbox and node_upper_bbox
        if not intersects:
            continue
        out.append(data[i])

    return out


# @profile
def search(tuple node, np.ndarray[DTFLT_t] bbox):
    items = []
    search_node(node, bbox, items)
    if len(items):
        iboxes,data = zip(*items)
    else:
        iboxes = [[]]
        data = []

    iboxes = np.asarray(iboxes)
    data = np.asarray(data).astype(int)

    items = search_nodes(bbox, iboxes, data)
    return items


In [24]:
root = r._root
print(type(root))

box = np.asarray([-100,-100,-50,-50]).astype(float)
print(type(box))

search(root, box)

<class 'tuple'>
<class 'numpy.ndarray'>


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,


In [25]:
def searchme(root, dt):
    for i in range(len(dt)):
        search(root, dt[i])

import cProfile
import pstats

# import pyximport
# pyximport.install()

cProfile.runctx('searchme(root, dt)', globals(), locals(), 'profile.prof')

s = pstats.Stats('profile.prof')
s.strip_dirs().sort_stats('time').print_stats()

Thu Feb  8 00:17:16 2018    profile.prof

         620144 function calls (7005 primitive calls) in 6.059 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    2.551    0.003    2.551    0.003 _cython_magic_98e173376efbe5cd0a9c0e07a3012a21.pyx:50(search_nodes)
     1000    1.288    0.001    6.029    0.006 _cython_magic_98e173376efbe5cd0a9c0e07a3012a21.pyx:86(search)
     2000    1.220    0.001    1.220    0.001 {built-in method numpy.core.multiarray.array}
614139/1000    0.967    0.000    0.967    0.001 _cython_magic_98e173376efbe5cd0a9c0e07a3012a21.pyx:14(search_node)
        1    0.030    0.030    6.059    6.059 <ipython-input-25-1260bc0808fa>:1(searchme)
     2000    0.003    0.000    1.223    0.001 numeric.py:463(asarray)
        1    0.000    0.000    6.059    6.059 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    6.059    

In [26]:
#Print profiling statistics using the `line_profiler` API
profile = line_profiler.LineProfiler(searchme)

profile.add_module(rbush.core_search)
profile.runcall(searchme, root, dt)
# profile.runctx('searchme(dt)', globals(), locals())
profile.print_stats()

Timer unit: 1e-06 s

Total time: 8.33489 s
File: <ipython-input-25-1260bc0808fa>
Function: searchme at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def searchme(root, dt):
     2      1001         1900      1.9      0.0      for i in range(len(dt)):
     3      1000      8332986   8333.0    100.0          search(root, dt[i])

